In [30]:
import pandas as pd

# Import data into dataframes, and drop unnecessary values
data_part1 = pd.read_csv('./dataset/Data_part1.csv')
data_part1Stage2 = pd.read_csv('./dataset/Data_part1Stage2.csv')
all_dataset_names = [data_part1, data_part1Stage2]
complete_dataset = pd.concat(all_dataset_names, ignore_index=True, sort=False).drop(columns=['Unnamed: 0'])
complete_dataset = complete_dataset.drop(columns=list(complete_dataset.filter(regex = '^cf')))
complete_dataset = complete_dataset.dropna(subset=['summary', 'depends_on'])
complete_dataset = complete_dataset.set_index('id')

C:\Users\hanna\AppData\Local\Temp\ipykernel_21004\1735775918.py:4: DtypeWarning: Columns (8,9,11,12,14,15,16,17,19,21,23,24,31,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,79,93,104,113,117,118,119,120,121,122,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,153,155,156,157,159,160,161,162,163,164,165,166,167,168,169,170,171,183,184,187,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,228,229,230,231,232,233,234,238,239,240,244,245,246,247,248,249,250,251,255,256,257,262,263,264,265,266,278) have mixed types. Specify dtype option on import or set low_memory=False.
  data_part1 = pd.read_csv('./dataset/Data_part1.csv')
C:\Users\hanna\AppData\Local\Temp\ipykernel_21004\1735775918.py:5: DtypeWarning: Columns (7,8,10,15,16,18,25,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,

In [31]:
#Get all of the rows which depend on something
index_dataset = complete_dataset[['depends_on']]
index_dataset = index_dataset.dropna()

#If you have multiple values in one cell, split them into different columns (Ex: 2 items seperated by a comma = 2 columns)
targets = index_dataset['depends_on'].str.split(',', expand=True)
#Count the number of instances in each row
result = targets.apply(pd.Series.value_counts)
#Remove NaN values - they throw things off
result = result.fillna(0)

#Sum all the columns, based on row. This will give us the total count for how many entries 
#are dependent on another, with the index being the ID of the "depends_on" entry
result = result.sum(axis='columns').to_frame()

#Find the entries which have the most references in the "depends_on" column
result = result.sort_values(by = 0, ascending=False)

# Give the top 150 referenced entries
result = result.head(150)

#Make a list of the entries
indicies = result.index.values.tolist()

#Make sure the entries are in string format, with all whitespace stripped
indicies = list(map(str,indicies))
indicies = [i.strip() for i in indicies]
print(indicies)

['1395112', '1130266', '1529362', '1445197', '1448703', '1575284', '1574241', '278458', '1470432', '1231208', '1471622', '1444991', '1319773', '1172897', '1464426', '1472491', '1320475', '1461360', '1574159', '1460343', '1477783', '231429', '1436478', '1511073', '1488620', '1459209', '1317102', '1322630', '1457891', '1496671', '1549775', '1452845', '1478118', '1444489', '1205323', '146679', '1461793', '1350424', '1446830', '1485081', '1408053', '1450956', '1217996', '1344091', '106592', '1218018', '1062849', '1464311', '1457988', '1457891', '1459555', '1460805', '260839', '132035', '1424281', '1574159', '1356397', '1483483', '1427928', '1446101', '1473772', '125419', '1353360', '1512802', '1248897', '1480504', '1464828', '1444489', '1475189', '1402247', '1467221', '1477676', '910412', '1471625', '231062', '57805', '1478372', '675539', '1466549', '1439588', '1467949', '1342051', '1482682', '1456680', '17483', '1191460', '1476876', '1473921', '1388019', '1446168', '1441308', '1418995', '

In [32]:
# Preprocessing
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

complete_dataset = complete_dataset[['depends_on', 'summary']]

#Used to remove the stopwords
def removeStopWords(startingList):
    returnedList = [x for x in startingList if x not in stopwords.words('english')]
    return returnedList

#Used to remove punctuation from the summary column
tokenizer = RegexpTokenizer(r'\w+')

summary_features = complete_dataset['summary']
summary_features = summary_features.to_frame()
#Remove punctuation
summary_features = summary_features.apply(lambda x: tokenizer.tokenize(x['summary']), axis=1)
#Remove stopwords
summary_features = summary_features.apply(lambda x: removeStopWords(x))

# Stemming + Standardization (Lemmatization)
lemmatizer = WordNetLemmatizer()
def applyLemmatization(startingList):
    returnedList = [lemmatizer.lemmatize(x) for x in startingList]
    return returnedList
summary_features = summary_features.apply(lambda x: applyLemmatization(x))
print(summary_features)

#Get the "depends_on" column into its own dataframe, change each cell to hold a list of strings (each ID = 1 string) instead of one string
targets = complete_dataset['depends_on'].str.split(',').to_frame()
#Replace all NaN values with empty lists
targets['depends_on'] = targets['depends_on'].apply(lambda x: x if isinstance(x, list) else [])
summary_features = summary_features.to_frame()

print(targets)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


id
1441804    [Learn, More, sumo, page, Cookies, Site, Data,...
1441827    [WebRender, initialization, failed, Intel, IGP...
1310761    [meta, Implement, grouped, session, history, a...
1179734     [meta, Responsive, Design, experiment, Whistler]
1310831    [DateTimeInput, Allow, copy, paste, value, tim...
                                 ...                        
1212765      [Use, property, SVG, attribute, summary, table]
1212767    [Use, property, CSS, summary, table, en, US, p...
1474961    [Change, StumblerService, foreground, service,...
1474968    [Font, rendering, sometimes, different, Window...
1474980    [Remove, unused, DebuggerServer, addActors, me...
Length: 6292, dtype: object
                                                depends_on
id                                                        
1441804                                          [1455351]
1441827                                          [1442093]
1310761  [1310770,  1320385,  1320735,  1321688,  13236...
117

In [33]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

#Change the summary dataframe to onehot encoding - each row can have more than one "1" because sentences have multiple words
mlb = MultiLabelBinarizer()
features = summary_features.join(pd.DataFrame(mlb.fit_transform(summary_features.pop(0)),columns=mlb.classes_,index=summary_features.index))
print(features)
#Change the depends_on dataframe to onehot encoding - each row can have more than one "1" because you can have more than one dependency
targets = targets.join(pd.DataFrame(mlb.fit_transform(targets.pop('depends_on')),columns=mlb.classes_,index=targets.index))

#Get rid of whitespace in the column names
targets.columns = targets.columns.str.strip()
targets


         0  00  008  01  010  011  017  0182  02  027  ...  zh  zombification  \
id                                                     ...                      
2654     0   0    0   0    0    0    0     0   0    0  ...   0              0   
2800     0   0    0   0    0    0    0     0   0    0  ...   0              0   
2875     0   0    0   0    0    0    0     0   0    0  ...   0              0   
2892     0   0    0   0    0    0    0     0   0    0  ...   0              0   
2892     0   0    0   0    0    0    0     0   0    0  ...   0              0   
...     ..  ..  ...  ..  ...  ...  ...   ...  ..  ...  ...  ..            ...   
1575575  0   0    0   0    0    0    0     0   0    0  ...   0              0   
1575624  0   0    0   0    0    0    0     0   0    0  ...   0              0   
1575639  0   0    0   0    0    0    0     0   0    0  ...   0              0   
1575708  0   0    0   0    0    0    0     0   0    0  ...   0              0   
1575726  0   0    0   0    0

,1000077,1000199,1000317,1000370,1000462,1000745,1000775,1000814,1000870,1000879,...,992569,9942,995252,996174,997190,997779,997906,998343,998494,999754
id,,,,,,,,,,,,,,,,,,,,,
2654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575624,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
#Save the list of all target values
all_indicies = targets.index.values.tolist()
full_target_list = targets

#Select only the columns which have entries in the top 150 dependencies
targets = targets[indicies]
#Get rid of duplicate rows and columns
targets = targets.groupby(level=0, axis=1).sum()
targets = targets.groupby(level=0, axis=0).sum()
#Get rid of rows which no longer have any dependencies (ones that depended on nothing in the top 150)
targets = targets.loc[(targets != 0).any(axis=1)]
targets

,103843,104960,1062849,106592,1073717,1130266,1172897,1191460,1205323,1217996,...,231429,252848,260839,278458,453908,547004,57805,672244,675539,910412
id,,,,,,,,,,,,,,,,,,,,,
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
#Get the indicies of the selected target values
target_indicies = targets.index.values.tolist()

#Remove duplicate rows
features = features.groupby(level=0, axis=0).sum()
#Select features which match the target indicies
features = features.loc[target_indicies]
#Remove columns which are all 0s
features = features.loc[:, features.any()]

features

,0,008,01,010,011,1,10,113,14,1442915,...,xpcom,xperf,xul,yaml,yosemite,zero,zh,zombification,zoom,zooming
id,,,,,,,,,,,,,,,,,,,,,
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29106,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#Make sure all values are either 1 or 0
def set_one_or_zero(x):
    if x == 0 or x == 1:
        return
    else:
        return (x/x)

targets.applymap(set_one_or_zero)
features.applymap(set_one_or_zero)

targets

,103843,104960,1062849,106592,1073717,1130266,1172897,1191460,1205323,1217996,...,231429,252848,260839,278458,453908,547004,57805,672244,675539,910412
id,,,,,,,,,,,,,,,,,,,,,
10097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1575282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_fscore_support

random_forest = MultiOutputClassifier(RandomForestClassifier())
naive_bayes = MultiOutputClassifier(MultinomialNB())

# Get predictions
# Split the training and test data (10% training)
X_train, X_test, y_train, y_test = train_test_split(features, targets, train_size=0.50)

# Fit models
random_forest.fit(X_train, y_train)
naive_bayes.fit(X_train, y_train)

# Predict for each given model
random_forest_pred = random_forest.predict(X_test)
naive_bayes_pred = naive_bayes.predict(X_test)

# Ensemble the results
# Since traditional ensemble models don't work with multi-output classifiers, we have to take the mean ourselves
Y_pred_chains = np.array([random_forest_pred, naive_bayes_pred])
Y_pred_ensemble = Y_pred_chains.mean(axis=0)

#Changes our results of y_predict & y_test into a format which the f1_score function will accept
binarizer = MultiLabelBinarizer().fit(y_test)

Y_test = binarizer.fit_transform(y_test.values.tolist())
Y_pred_ensemble = binarizer.transform(Y_pred_ensemble)

# Print scores
metrics = precision_recall_fscore_support(Y_test, Y_pred_ensemble, average='weighted')
print("F1 score: {}".format(metrics[2]))
print("Precision score: {}".format((metrics[0])))
print("Recall-score: {}".format((metrics[1])))


F1 score: 0.7315286993397432
Precision score: 0.793922738350236
Recall-score: 0.6982142857142857


c:\Users\hanna\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) [0.5] will be ignored
  warnings.warn(
